In [1]:
# Importing necessary libraries 
import sys
import os
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Appending the parent directory (root of the project) to the system path
sys.path.append(os.path.abspath('../'))

In [2]:
# Import custom functions from the src package
from src import create_image_mask

c:\Users\Tyler\anaconda3\envs\bp_ai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load a sample image using OpenCV
sample_image_path = '../data/base_images/warehouse/warehouse_0000.jpg'  # Replace with the actual path
sample_image = cv2.imread(sample_image_path)
sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)  # Convert to RGB for visualization

# Create an image mask using the custom mask generation function
mask, bboxes = create_image_mask(sample_image)

# Display the original image and the mask
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(sample_image)
plt.title('Original Image')
plt.subplot(1, 2, 2)
plt.imshow(mask, cmap='gray')
plt.title('Generated Mask')
plt.show()

In [ ]:
# Run the demo with default parameters and config file
# TODO THIS DOESN'T WORK RIGHT NOW 12/21/23
!python ../src/inpainting_dataset_generator.py

In [ ]:
# Below are a few functions to help visualize the results of dataset generation
from PIL import Image, ImageDraw

# Function to read the bounding box annotations from the generated text file
def read_bounding_boxes(file_path, image_width, image_height):
    bounding_boxes = []
    with open(file_path, "r") as file:
        for line in file:
            # Extracting YOLO format coordinates
            _, x_center, y_center, width, height = map(float, line.strip().split())

            # Convert from YOLO format (center) to corner coordinates
            x_min = (x_center - width / 2) * image_width
            y_min = (y_center - height / 2) * image_height
            x_max = (x_center + width / 2) * image_width
            y_max = (y_center + height / 2) * image_height

            bounding_boxes.append((x_min, y_min, x_max, y_max))

    return bounding_boxes

# Function to draw the bounding boxes on the inpainted image
def draw_bounding_boxes(image, bounding_boxes):
    draw = ImageDraw.Draw(image)
    for box in bounding_boxes:
        draw.rectangle(box, outline="red", width=2)
    return image

# Function that handles the reading and drawing of bounding boxes on an image
def overlay_bboxes(image_path, bbox_file_path):
    # Read the image
    image = Image.open(image_path)
    image_width, image_height = image.size

    # Read bounding boxes from file, scaled to image dimensions
    bounding_boxes = read_bounding_boxes(bbox_file_path, image_width, image_height)

    # Draw bounding boxes on the image
    image_with_boxes = draw_bounding_boxes(image, bounding_boxes)

    # Return the result
    return image_with_boxes

In [ ]:
# Visualizing the inpainted image with the generated bounding box label
image_path = "../data/output/example_dataset_demo/warehouse_0000_0_0001.png"
bbox_file_path = "../data/output/example_dataset_demo/warehouse_0000_0_0001.txt"
output_path = "path_to_save_output_image.jpg"

labeled_image = overlay_bboxes(image_path, bbox_file_path)
labeled_image